In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

H5Nri8wyhkcbyDiw2b_z


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json
import numpy as np

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-02&end_date=2017-01-02&api_key=H5Nri8wyhkcbyDiw2b_z"
r = requests.get(url)
data = r.json()['dataset_data']
data


{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'start_date': '2017-01-02',
 'end_date': '2017-01-02',
 'frequency': 'daily',
 'data': [['2017-01-02',
   34.99,
   35.94,
   34.99,
   35.8,
   None,
   44700.0,
   1590561.0,
   None,
   None,
   None]],
 'collapse': None,
 'order': None}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#Get data
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=H5Nri8wyhkcbyDiw2b_z"
r = requests.get(url)
data = r.json()['dataset_data']

In [6]:
#Check that JSON object properly loaded as a Python dictionary
print(type(data))

<class 'dict'>


In [7]:
#Parse data into a list containing only date and opening price

def parse_data(index):
    dataset = data.get('data')
    index_data_list = []
    for item in dataset:
        if item[index] is not None:
            index_data_list.append(item[index])
    return index_data_list

In [8]:
#Calculate max opening price and min opening price with associated date
price = parse_data(1)
print("The highest opening price in this period was a price of $ " + str(max(price)))
print("The lowest opening price in this period was a price of $ " + str(min(price)))

The highest opening price in this period was a price of $ 53.11
The lowest opening price in this period was a price of $ 34.0


In [9]:
# Calculate the largest change in any one day (based on High and Low price)
high = np.array(parse_data(2))
low = np.array(parse_data(3))
delta = np.subtract(high, low)
print("The largest change in any one day was: $" + "{:.2f}".format(max(delta)))

The largest change in any one day was: $2.81


In [10]:
#Calculate the largest change between any two days (based on Closing Price)
closing_price = parse_data(4)
diff_price = []
for x in range(1,len(closing_price)):
    diff = closing_price[x] - closing_price[x-1]
    diff_price.append(diff)

print("The largest change in closing price between any two consecutive days is: $ " +  "{:.2f}".format(max(diff_price)) )

The largest change in closing price between any two consecutive days is: $ 2.56


In [11]:
#Calculate the average daily trading volume in 2017
daily_trading_volume = parse_data(6)
np_daily_tv = np.array(daily_trading_volume)
print("The average daily trading volume in 2017 was: " + "{:.2f}".format(np.average(np_daily_tv)))

The average daily trading volume in 2017 was: 89124.34


In [12]:
#Calculate the median trading volume in 2017
print("The average daily trading volume in 2017 was: " + "{:.2f}".format(np.median(np_daily_tv)))

The average daily trading volume in 2017 was: 76286.00
